In [4]:
from sentence_transformers import SentenceTransformer, util
import json
import glob
import spacy
import numpy as np


e:\Codigo\python\Hugging-Face\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cargar el modelo de SentenceTransformer
model = SentenceTransformer("NovaSearch/stella_en_1.5B_v5", trust_remote_code=True)

# Cargar el modelo de spaCy para el idioma español
nlp = spacy.load("es_core_news_sm")


In [9]:
# Ruta de la carpeta que contiene los archivos JSON
carpeta_fichas = "./fichas"

# Lista para almacenar las oraciones extraídas
sentences = []

# Recorrer todos los archivos JSON en la carpeta
for archivo in glob.glob(f"{carpeta_fichas}/*.json"):
    # Leer el contenido del archivo JSON
    with open(archivo, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
        # Extraer valores de las claves especificadas si existen
        id_ficha = str(data.get('IdFichaTecnica', ''))
        codigo_pelicula = data.get('codigoDePelicula', '')
        aplicaciones = data.get('Aplicaciones', '')
        
        # Crear una oración con los valores (puedes ajustar el formato)
        sentence = f"IdFichaTecnica: {id_ficha}, CodigoDePelicula: {codigo_pelicula}, Aplicaciones: {aplicaciones}"
        sentences.append(sentence)


In [10]:
# Generar los embeddings de las oraciones SIN oversampling
sentence_embeddings = model.encode(sentences)

In [2]:
# ---- Función para "sobremuestrear" (oversampling) los sustantivos SOLO en la query ----
def ponderar_sustantivos(oracion, factor=2.0):
    doc = nlp(oracion)
    palabras_ponderadas = []
    for token in doc:
        if token.pos_ == "NOUN":  # Si la palabra es un sustantivo
            # Añadir el sustantivo varias veces (factor = 2.0 por defecto)
            palabras_ponderadas.extend([token.text] * int(factor))
        else:
            palabras_ponderadas.append(token.text)
    return " ".join(palabras_ponderadas)

In [3]:
# La oración de referencia
reference_sentence = "envolver o empacar carnes o atun,"

# Oversampling SOLO en la frase de referencia
reference_sentence_ponderada = ponderar_sustantivos(reference_sentence, factor=3)

# Generar el embedding de la frase de referencia sobresampleada
reference_embedding = model.encode(reference_sentence_ponderada)



NameError: name 'nlp' is not defined

In [28]:
# Calcular la similitud coseno entre la oración de referencia y cada oración en la lista
similarities = util.cos_sim(reference_embedding, sentence_embeddings)
# 4. Convertir el tensor a una lista para una salida más legible
similarities = similarities.tolist()[0]  # Convertir a lista y tomar la primera fila

In [29]:
# 5. Mostrar los resultados
print("Similitudes entre la oración de referencia y las oraciones en la lista:")

# Crear una lista de tuplas (índice, similitud) y ordenarla por similitud de mayor a menor
sorted_similarities = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)

# Imprimir los resultados ordenados
for rank, (i, sim) in enumerate(sorted_similarities, start=1):
    print(f"\n{rank}) Sentence: '{sentences[i]}' \n   Score: {sim:.4f}")

Similitudes entre la oración de referencia y las oraciones en la lista:

1) Sentence: 'IdFichaTecnica: , CodigoDePelicula: ATr-TT, Aplicaciones: Grado retortable, se utiliza como capa intermedia en laminaciones triples o cuádruples para envases flexibles retortables que deben soportar exigentes condiciones de esterilización. Estos envases reemplazan latas de conservas y se usan para empacar atún, pescados, comidas preparadas y otros.Cumple con las regulaciones de la FDA y UE para contacto con alimentos.' 
   Score: 0.4848

2) Sentence: 'IdFichaTecnica: , CodigoDePelicula: ATr, Aplicaciones: Grado retortable, se utiliza como capa intermedia en laminaciones triples o cuádruples para envases flexibles retortables que deben soportar exigentes condiciones de esterilización. Estos envases reemplazan latas de conservas y se usan para empacar atún, pescados, comidas preparadas y otros. Cumple con las regulaciones de la FDA y UE para contacto con alimentos.' 
   Score: 0.4837

3) Sentence: 'IdF

In [32]:
from sentence_transformers import CrossEncoder
model_cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1', max_length=512)
# Tomamos el TOP-15 de la primera etapa
top_15_indices = [idx for (idx, sim) in sorted_similarities[:15]]

# Construimos los pares (query, oración) para el cross-encoder
pairs = [(reference_sentence, sentences[i]) for i in top_15_indices]

# El cross-encoder produce scores de relevancia
cross_scores = model_cross_encoder.predict(pairs)

In [33]:
# Reunimos índices y scores
# Reordenamos descendente
reranked = sorted(zip(top_15_indices, cross_scores), key=lambda x: x[1], reverse=True)

print("\n===== [SEGUNDO RANKING: Re-rank con cross-encoder MS MARCO] =====\n")
for rank, (idx, cscore) in enumerate(reranked, start=1):
    print(f"{rank}) Cross-Encoder Score: {cscore:.4f} | Sentence: '{sentences[idx]}'")


===== [SEGUNDO RANKING: Re-rank con cross-encoder MS MARCO] =====

1) Cross-Encoder Score: -1.9193 | Sentence: 'IdFichaTecnica: , CodigoDePelicula: ATr, Aplicaciones: Grado retortable, se utiliza como capa intermedia en laminaciones triples o cuádruples para envases flexibles retortables que deben soportar exigentes condiciones de esterilización. Estos envases reemplazan latas de conservas y se usan para empacar atún, pescados, comidas preparadas y otros. Cumple con las regulaciones de la FDA y UE para contacto con alimentos.'
2) Cross-Encoder Score: -2.0606 | Sentence: 'IdFichaTecnica: , CodigoDePelicula: AT, Aplicaciones: Diseñada como película externa, impresa reversa en laminados. Utilizada en empaques al vacío o en atmósfera modificada que requiera buena barrera al oxígeno, para carnes procesadas, embutidos, mariscos, congelados, productos lácteos y pastas. También se usa en paquetes que requieren una excelente resistencia mecánica, resistencia al punzonado y “flexcrack”. Además 